In [1]:
import modulos as md
import sys
import os
import constantes as cons
import numpy as np 
import pandas as pd
symbol='STXUSDT'

lista=md.lista_de_monedas()
for symbol in lista:
    try:
        data = md.obtiene_historial(symbol)
        data = md.estrategia_bb(data)
        resultado = md.backtesting(data)
        if resultado[6]>1.5:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]}")
    except Exception as falla:
        pass  


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EOSUSDT - retorno: 4.13% - trades: 6
XLMUSDT - retorno: 1.76% - trades: 5
XTZUSDT - retorno: 2.32% - trades: 7
THETAUSDT - retorno: 7.37% - trades: 7
ZRXUSDT - retorno: 2.16% - trades: 5

Error: APIError(code=-1121): Invalid symbol. - line: 230 - file: modulos.py - par: DEFIUSDT



c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:304: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


RUNEUSDT - retorno: 13.52% - trades: 7
SUSHIUSDT - retorno: 7.84% - trades: 7
SRMUSDT - retorno: 2.25% - trades: 5
STORJUSDT - retorno: 5.66% - trades: 6
ENJUSDT - retorno: 4.02% - trades: 4
CTKUSDT - retorno: 2.12% - trades: 5
1INCHUSDT - retorno: 2.37% - trades: 8
MANAUSDT - retorno: 1.91% - trades: 14
ALICEUSDT - retorno: 2.43% - trades: 12

Error: APIError(code=-1121): Invalid symbol. - line: 230 - file: modulos.py - par: 1000SHIBUSDT


Error: APIError(code=-1121): Invalid symbol. - line: 230 - file: modulos.py - par: BTCDOMUSDT


Error: APIError(code=-1121): Invalid symbol. - line: 230 - file: modulos.py - par: 1000XECUSDT

CELOUSDT - retorno: 4.64% - trades: 5

Error: APIError(code=-1121): Invalid symbol. - line: 230 - file: modulos.py - par: FOOTBALLUSDT


Error: APIError(code=-1121): Invalid symbol. - line: 230 - file: modulos.py - par: 1000LUNCUSDT


Error: APIError(code=-1121): Invalid symbol. - line: 230 - file: modulos.py - par: LUNA2USDT


Error: APIError(code=-1121): Inva